In [83]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

# English Translated

In [84]:
data=pd.read_pickle('C:/Users/forgo/Desktop/dsa4266/full_df_en_processed.pkl').reset_index()#.drop_duplicates('clean_msg')
temp=pd.read_pickle('C:/Users/forgo/Desktop/dsa4266/full_df.pkl').reset_index()
data=data.merge(temp[['doc_id','pretranslation']],how='left',on='doc_id').drop_duplicates('pretranslation').drop_duplicates('clean_msg')
# data=data[['class','processed']].drop_duplicates().rename(columns={'processed':'clean_msg'})
data['lang']=data['doc_id'].apply(lambda x: x[6])
data=data[data['clean_msg']!=''].reset_index(drop=True)
data

,doc_id,class,translated,clean_msg,pretranslation,lang
0,trec06c/data/000/007,spam,Dear person in charge (manager/finance): Hello...,dear person charge hello trading co ltd strong...,尊敬的负责人（经理／财务）：您好！ 我是深圳伟仕嘉贸易有公司：兴办贸易、物资供销，实力雄厚；...,c
1,trec06c/data/000/014,spam,Hello! Nice to meet you. Our company is intere...,hello nice meet company interested cooperating...,"您好！很高兴认识您。我司有意与您们合作:可长久给您们带来 巨大的效益,另因合作项目的高度自动...",c
2,trec06c/data/000/005,spam,TO: Your company’s manager and finance manager...,manager finance manager hello trading co ltd b...,TO：贵公司经理、财务 您好！ 深圳市春洋贸易有限公司（东莞分公司）...,c
3,trec06c/data/000/013,spam,New remote-controlled airplanes are widely sol...,new airplanes widely sold many types toy airpl...,新型遥控飞机销路广 百元可办厂 玩具飞机品种繁多，但用微型小开关控制的新型能在空中自控飞翔的...,c
4,trec06c/data/000/021,spam,"Dear company (factory) manager, hello: Our com...",dear company factory manager hello company com...,尊敬的公司（工厂）经理负责人你好： 我公司是一家多年为外资企业代理进出口业务的公司，现有部分...,c
...,...,...,...,...,...,...
39240,trec06p/data/035/115,spam,Rent WITHOUT COMMISSION\r \r m. Avtozavodskaya...,rent without commission min foot blocks buildi...,"Аренда БЕЗ КОМИССИОННЫХ\r \r м. Автозаводская,...",p
39241,trec06p/data/058/203,ham,RIFFx~ WAVEfmt + + dataS~ ~y{yw{ zuvx}}ytmjhed...,aa aa tr pf en kn la qu oka pk oz gun nom mum ...,RIFFx~ WAVEfmt      +  +    dataS~ ...,p
39242,trec06p/data/014/167,spam,dangerous cork blob thirteenth alliterate argu...,dangerous cork blob thirteenth alliterate argu...,dangerous cork blob thirteenth alliterate argu...,p
39243,trec06p/data/025/231,spam,:*.☆. o:☆';*. :*.☆. o:☆';*. :*.☆. o:☆';*. :*.☆...,largest online community history born site abs...,\r :*.☆。o:☆';*。:*.☆。o:☆';*。:*.☆。o:☆';*。:*.☆...,p


In [85]:
data.groupby('class').count()

,doc_id,translated,clean_msg,pretranslation,lang
class,,,,,
ham,30579,30579,30579,30579,30579
spam,8666,8666,8666,8666,8666


In [86]:
# en_regex = re.compile('[^a-zA-Z ]')
# d = enchant.Dict("en_US") 
# STOPWORDS = stopwords.words('english')

In [87]:
# def enPreprocess(string):
#     string=string.replace('\n',' ') #remove newline char
#     string=en_regex.sub('',string) #removes non alphabets
#     string=string.split()
#     string=[word.lower() for word in string if len(word)>1]
#     string=' '.join([word for word in string if d.check(word) and (word not in STOPWORDS)]) # split the string into list and check each word if it is in the english dictionary and longer than 1 alphabet
#     return string

In [88]:
# data['clean_msg']=data['translated'].apply(lambda x: enPreprocess(x))
# data.to_pickle('lowercase words.pkl')

In [89]:

X = data.drop(columns='class')
y = data['class']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.2,random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,test_size=.2,random_state=42)

In [90]:
y_train.value_counts()

ham     19566
spam     5550
Name: class, dtype: int64

In [91]:
# y_train=pd.concat([y_train[y_train=='ham'].sample(5528),y_train[y_train=='spam']]).sample(frac=1) #under sampling
# y_train=pd.concat([y_train[y_train=='ham'],y_train[y_train=='spam'].sample(24461,replace=True)]).sample(frac=1) #over sampling

In [92]:
X_train=X_train.loc[y_train.index]

In [93]:
X_train_e=X_train[X_train['lang']=='p']['clean_msg']
X_val_e=X_val[X_val['lang']=='p']['clean_msg']
X_test_e=X_test[X_test['lang']=='p']['clean_msg']
y_train_e=y_train.loc[X_train_e.index]
y_val_e=y_val.loc[X_val_e.index]
y_test_e=y_test.loc[X_test_e.index]

X_train_c=X_train[X_train['lang']=='c']['pretranslation']
X_val_c=X_val[X_val['lang']=='c']['pretranslation']
X_test_c=X_test[X_test['lang']=='c']['pretranslation']
y_train_c=y_train.loc[X_train_c.index]
y_val_c=y_val.loc[X_val_c.index]
y_test_c=y_test.loc[X_test_c.index]

X_train=X_train['clean_msg']
X_val=X_val['clean_msg']
X_test=X_test['clean_msg']

In [94]:
vect = CountVectorizer(stop_words='english')
X_train_dtm = vect.fit_transform(X_train)
X_val_dtm = vect.transform(X_val)
X_test_dtm= vect.transform(X_test)


In [95]:
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_val_dtm)

print('translated dataset Multinomial Naive Bayes ')
print('acc',metrics.accuracy_score(y_val, y_pred_class))
print('f1',f1_score(y_val.to_list(), y_pred_class,pos_label="spam"))
# metrics.confusion_matrix(y_test, y_pred_class)

translated dataset Multinomial Naive Bayes 
acc 0.9320063694267516
f1 0.8406121687196716


In [96]:
logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train_dtm, y_train)
y_pred_class = logreg.predict(X_val_dtm)
y_pred_prob = logreg.predict_proba(X_val_dtm)[:, 1]

print('translated dataset log regression ')
print('acc',metrics.accuracy_score(y_val, y_pred_class))
print('validation f1',f1_score(y_val.to_list(), y_pred_class,pos_label="spam"))

translated dataset log regression 
acc 0.9684713375796178
validation f1 0.9283646888567294


C:\Users\forgo\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [97]:
a=[]
b=[]
for i in np.arange(0.7,.85,0.001):
    a.append(i)
    c=pd.Series(y_pred_prob).apply(lambda x: 'spam' if x>i else 'ham')
    b.append(f1_score(y_val.to_list(),c,pos_label="spam"))


In [98]:
thresh_hold_table=pd.DataFrame({'threshhold value':a,'f1_score':b}).sort_values('f1_score',ascending=False).head(10)
thresh_hold_table

,threshhold value,f1_score
0,0.700,0.916163
1,0.701,0.915690
2,0.702,0.915690
3,0.703,0.915690
4,0.704,0.915690
5,0.705,0.915690
6,0.706,0.915690
15,0.715,0.915562
16,0.716,0.915562
9,0.709,0.915280


In [99]:
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
opt_predicted=pd.Series(y_pred_prob).apply(lambda x: 'spam' if x>thresh_hold_table.iloc[0,0] else 'ham')
print('test f1',f1_score(y_test.to_list(), opt_predicted,pos_label="spam"))

test f1 0.9136994568497284


# 1 model for each language

In [100]:
vect = CountVectorizer(stop_words='english')
X_train_e_dtm = vect.fit_transform(X_train_e)
X_test_e_dtm= vect.transform(X_test_e)

In [101]:
import re
cn_word=re.compile("[\u4e00-\u9FFF]")
X_train_c=X_train_c.apply(lambda x: ''.join([word for word in x if cn_word.match(word)]))
X_test_c=X_test_c.apply(lambda x: ''.join([word for word in x if cn_word.match(word)]))

In [102]:
vect = CountVectorizer(tokenizer=lambda txt:[*txt])
X_train_c_dtm = vect.fit_transform(X_train_c)
X_test_c_dtm= vect.transform(X_test_c)

C:\Users\forgo\AppData\Roaming\Python\Python310\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [103]:

nb = MultinomialNB()
nb.fit(X_train_e_dtm, y_train_e)
y_pred_e_class = nb.predict(X_test_e_dtm)


nb = MultinomialNB()
nb.fit(X_train_c_dtm, y_train_c)
y_pred_c_class = nb.predict(X_test_c_dtm)

In [104]:
print('2 model combined Multinomial Naive Bayes ')
y_pred_combined=y_pred_e_class.tolist()+y_pred_c_class.tolist()
y_actual_combined=pd.concat([y_test_e,y_test_c]).to_list()
print('acc',metrics.accuracy_score(y_actual_combined, y_pred_combined))
print('f1',f1_score(y_pred_combined,y_actual_combined,pos_label="spam"))

2 model combined Multinomial Naive Bayes 
acc 0.9371894508854631
f1 0.8630935851152458


In [105]:

logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train_e_dtm, y_train_e)
y_pred_e_class = logreg.predict(X_test_e_dtm)
# y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]

logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train_c_dtm, y_train_c)
y_pred_c_class = logreg.predict(X_test_c_dtm)
# y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]

In [106]:

print('2 model combined log regression ')
y_pred_combined=y_pred_e_class.tolist()+y_pred_c_class.tolist()
y_actual_combined=pd.concat([y_test_e,y_test_c]).to_list()
print('acc',metrics.accuracy_score(y_actual_combined, y_pred_combined))
print('f1',f1_score(y_pred_combined,y_actual_combined,pos_label="spam"))

2 model combined log regression 
acc 0.9770671423111225
f1 0.9477958236658932
